# Introduction to FEM Analysis with Python

This tutorial aims to show using Python to pre-processing, solve, and post-processing of Finite Element Method analysis. It uses a finite element method library with a Python interface called [GetFEM](http://getfem.org/) for preprocessing and solving. We will load vtk file by using [meshio](https://github.com/nschloe/meshio) and visualize by [matplotlib](https://github.com/matplotlib/matplotlib) in pre-processing and post-processing. This tutorial was used in the [PyConJP 2019 talk](https://pycon.jp/2019/schedule/?sessionId=120). You can watch the talk on YouTube below. The tutorial descriptions are heavily influenced by the [official GetFEM pages](http://getfem.org/tutorial/thermo_coupling.html).

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo("6JuB1GiDLQQ", start=512)

## Installation

GetFEM including its python interface can be installed from a terminal by executing aptitude update and aptitude install python-getfem++.

The additional packages in [requirements.txt](https://github.com/getfem-doc/getfem-binder/blob/master/requirements.txt) are required for this tutorial. You do not need to build these environments because they are already configured in the [Dockerfile](https://github.com/getfem-doc/getfem-binder/blob/master/Dockerfile).

## The problem setting

The problem refers to ["Poisson's Equation on Unit Disk"](https://mathworks.com/help/pde/ug/solve-poissons-equation-on-a-unit-disk.html) published by Math Works's homepage.


$$−\Delta u=1 \ {\rm on}\  \Omega, u=0 \ {\rm on}\  \delta \Omega $$

![pdedemo1_01](https://jp.mathworks.com/help/examples/pde/win64/pdedemo1_01.png)



## How to use GetEM

We take the following steps when using GetFEM to solve finite element problems. See [this page](http://getfem.org/tutorial/basic_usage.html) for more information on using GetFEM.

- define a [MesherObject](http://getfem.org/python/cmdref_MesherObject.html)
- define a [Mesh](http://getfem.org/python/cmdref_Mesh.html)
- define a [MeshFem](http://getfem.org/python/cmdref_MeshFem.html)
- define a [MeshIm](http://getfem.org/python/cmdref_MeshIm.html)
- define a [Model](http://getfem.org/python/cmdref_Model.html) and set it up
- solve Model object
- get value from Model object

## Initialization

GetFEM can be imported following (numpy has also to be imported). 

In [ ]:
import getfem as gf
import numpy as np

## Mesh generation

We use GetFEM's [MesherObject](http://getfem.org/python/cmdref_MesherObject.html#) to create a mesh from the geometric information to be analyzed.
This object represents a geometric object to be meshed by the experimental meshing procedure of GetFEM. We can represents a ball of corresponding center and radius by using "ball".

In [ ]:
center = [0.0, 0.0]
radius = 1.0

mo = gf.MesherObject("ball", center, radius)

We can make mesh object `mesh` by calling the experimental mesher of GetFEM on the geometry represented by `mo`.

In [ ]:
h = 0.1
mesh = gf.Mesh("generate", mo, h, 2)

## Boundary selection

To define a boundary condition, we set a boundary number on the outer circumference of the circle.

In [ ]:
outer_faces = mesh.outer_faces()
OUTER_BOUND = 1
mesh.set_region(OUTER_BOUND, outer_faces)

## Mesh draw

We visualize the created mesh to check its quality. We make Slice of ject to create a mesh slice.We can output mesh objects, but matplotlib can only output triangles. Therefore, we convert mesh to triangles on the slice object.

In [ ]:
sl = gf.Slice(("none",), mesh, 1)

We can export a slice to VTK file by using export_to_vtk method.

In [ ]:
sl.export_to_vtk("sl.vtk", "ascii")

We can rendere VTK files using Paraview or mayavi2. In order to display in the jupyter notebook this time, we read in meshio and draw in matplotlib.

In [ ]:
import meshio

reader = meshio.read("sl.vtk")
points = reader.points
cells = reader.cells[0][1]

In [ ]:
%matplotlib inline
from matplotlib.pyplot import figure

figure = figure(figsize=(7, 7))
axes = figure.add_subplot(aspect="equal")
axes.triplot(points[:, 0], points[:, 1], cells, color="black")

## Definition of finite element methods and integration method

![Lagrange Element](https://raw.githubusercontent.com/tkoyama010/OpenCAE-tkoyama010/PyConJP2019/getfemlisttriangleP2.png)

In [ ]:
# The node has 1 degree of freedom
mfu = gf.MeshFem(mesh, 1)

# classical lagrange elements
elements_degree = 2
mfu.set_classical_fem(elements_degree)

In [ ]:
mim = gf.MeshIm(mesh, pow(elements_degree, 2))

## 🏃 [Model Object](http://getfem.org/userdoc/model_object.html)

$$ [K] \left\{ u \right\} = \left\{ F \right\} $$

In [ ]:
md = gf.Model("real")
md.add_fem_variable("u", mfu)

## 🏃 Laplacian Brick, RHS and condition 

$$−\Delta u=1 \ {\rm on}\  \Omega, u=0 \ {\rm on}\  \delta \Omega $$
to
$$[K] \left\{ u \right\} = \left\{ F \right\}$$

In [ ]:
md.add_Laplacian_brick(mim, "u")

In [ ]:
F = 1.0
md.add_fem_data("F", mfu)
md.add_source_term_brick(mim, "u", "F")
md.set_variable("F", np.repeat(F, mfu.nbdof()))

In [ ]:
md.add_Dirichlet_condition_with_multipliers(mim, "u", elements_degree - 1, OUTER_BOUND)

## 🏃 Solve $u$

$$[K] \left\{ u \right\} = \left\{ F \right\}$$

In [ ]:
md.solve()
U = md.variable("u")

## variable ***u***

In [ ]:
sl.export_to_vtk("u.vtk", "ascii", mfu, U, "U")
reader = meshio.read("u.vtk")
points = reader.points
cells = reader.cells[0][1]
point_data = reader.point_data["U"]

In [ ]:
from matplotlib.pyplot import figure

figure = figure(figsize=(7, 7))
axes = figure.add_subplot(aspect="equal")
axes.triplot(points[:, 0], points[:, 1], cells, color="gray")
contour = axes.tricontourf(
    points[:, 0], points[:, 1], cells, point_data, 16, cmap="jet"
)
figure.colorbar(contour)

## theoretical solution

The theoretical solution to this problem is given by the following equation:
$$u(x, y) = \dfrac{1-x^2-y^2}{4}$$

In [ ]:
evalue = mfu.eval("(1-x*x-y*y)/4")

We can calculate the error for the L2 and H1 norms by using the following functions:

In [ ]:
L2error = gf.compute(mfu, U - evalue, "L2 norm", mim)
H1error = gf.compute(mfu, U - evalue, "H1 norm", mim)
print("Error in L2 norm : ", L2error)
print("Error in H1 norm : ", H1error)